In [42]:
import pandas as pd
import numpy as np

print(pd.__version__)

1.4.3


In [43]:
#get the current table
GITHUB_URL = 'https://raw.githubusercontent.com/liampearson/nndss/main/nndss.csv'
table = pd.read_csv(GITHUB_URL)
table

,disease_group,disease_name,disease_code,act,nsw,nt,qld,sa,tas,vic,...,act_cumulative_sum,nsw_cumulative_sum,nt_cumulative_sum,qld_cumulative_sum,sa_cumulative_sum,tas_cumulative_sum,vic_cumulative_sum,wa_cumulative_sum,aus,aus_cumulative_sum
0,Vectorborne diseases,Flavivirus infection (unspecified),1,0.0,0.0,0,0,0,0.0,0.0,...,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.0
1,Vectorborne diseases,Flavivirus infection (unspecified),1,0.0,0.0,0,0,0,0.0,0.0,...,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.0
2,Vectorborne diseases,Flavivirus infection (unspecified),1,0.0,0.0,0,0,0,0.0,0.0,...,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.0
3,Vectorborne diseases,Flavivirus infection (unspecified),1,0.0,0.0,0,0,0,0.0,0.0,...,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.0
4,Vectorborne diseases,Flavivirus infection (unspecified),1,0.0,0.0,0,0,0,0.0,0.0,...,0.0,0.0,0,0,0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1497,Zoonoses,Monkeypox virus (MPXV) infection++,84,0.0,8.0,0,1,1,0.0,18.0,...,2.0,39.0,0,3,2,0.0,43.0,4,31.0,93.0
1498,Zoonoses,Monkeypox virus (MPXV) infection++,84,0.0,9.0,0,0,0,0.0,19.0,...,2.0,48.0,0,3,2,0.0,62.0,5,29.0,122.0
1499,Zoonoses,Monkeypox virus (MPXV) infection++,84,1.0,3.0,0,1,0,0.0,1.0,...,3.0,51.0,0,4,2,0.0,63.0,6,7.0,129.0
1500,Zoonoses,Monkeypox virus (MPXV) infection++,84,0.0,0.0,0,1,0,0.0,1.0,...,3.0,51.0,0,5,2,0.0,64.0,7,3.0,132.0


In [44]:
# when was the latest update?
table.end_date.max()

'2022-10-16'

In [62]:
urls = [

#17 to 30 Oct
#'https://www.health.gov.au/sites/default/files/2022-11/3._fortnight_22_-_07_nov2022_adt_-_web.xlsx',

#31 Oct to 13 Nov
'https://www.health.gov.au/sites/default/files/2022-12/3._fortnight_23_-_21_nov2022_adt_-web.xlsx',

#14 to 27 Nov
'https://www.health.gov.au/sites/default/files/2022-12/national_notifiable_diseases_surveillance_system_nndss_fortnightly_reports_-_14_to_27_november_2022.xlsx',

#
'https://www.health.gov.au/sites/default/files/2022-12/national_notifiable_diseases_surveillance_system_nndss_fortnightly_reports_-_28_november_to_11_december_2022.xlsx'

]

In [63]:
#iterate over each report/url/excel_file
#There's usually just one but have looped it incase I forget/miss some weeks.

for url in urls:
    #print("importing: ",url)
    # import the data
    df = pd.read_excel(url, header=2)
    
    df['ACT'] = df['ACT^^']

    #get the report start and end dates
    start_date = df['This reporting period'].iloc[0].strftime('%Y-%m-%d')
    end_date = df['This reporting period'].iloc[1].strftime('%Y-%m-%d')

    print("report for dates: {} --> {}; processing...".format(start_date, end_date))

    # DATA PREP ------------------
    #tidy up the header names: make all lower case
    df = df.rename(columns=str.lower)

    #replace spaces in header names with underscore
    df.columns = df. columns.str.replace(' ','_')

    #Get the index where the footnotes begin - we don't want these
    footnote_index = df[df['disease_group'].str.contains('Footnotes')==True].index.values[0]

    # Only keep the rows BEFORE the footnotes index
    # -1 to also remove the totals row
    df = df[df.index < footnote_index-1]
    #Forward fill the 'Disease group' labels (they're merged cells in excel)
    df['disease_group'] = df['disease_group'].fillna(method = 'ffill')

    #drop the date rows
    df.drop(index=[0,1], inplace=True)

    #convert disease code to int
    df['disease_code'] = df['disease_code'].astype(int)

    #convert report numbers to int

    # we just want the raw numbers, not the % change etc etc
    keep_headers = ['disease_group', 'disease_name', 'disease_code', 'act', 'nsw', 'nt',
                  'qld', 'sa', 'tas', 'vic', 'wa']

    # reduce dataframe to just the columns we want
    df = df[keep_headers]

    #add start and end dates
    df['start_date'] = start_date
    df['end_date'] = end_date

    # for each state, strip punctuation and convert the state numbers to INT & 
    states = ['act', 'nsw', 'nt','qld', 'sa', 'tas', 'vic', 'wa']

    #strip punctuation
    for state in states:
        # we need to remove all punctuation (like commas and asterisks), but that will cause 10.0 to go to 100 so:
        df[state] = df[state].astype(str).str.replace(".0", "", regex=False)

        #if df[state].dtype==object: #then we need to clean the numbers
        df[state] = df[state].astype(str).str.replace("[^\w\s]", "", regex=True)

        # replace NN (not notifiable) with nulls
        df[state].replace('NN', np.nan, inplace=True)

        # workaround since Int64 cannot convert objects
        df[state] = pd.to_numeric(df[state], errors='coerce').astype('Int64')

    # add this report to the rest of the data
    table = table.append(df)

print("report for period: {} --> {}; completed".format(start_date, end_date))

report for dates: 2022-10-31 --> 2022-11-13; processing...


<ipython-input-63-87eb14e9ce30>:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(df)
<ipython-input-63-87eb14e9ce30>:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(df)


report for dates: 2022-11-14 --> 2022-11-27; processing...
report for dates: 2022-11-28 --> 2022-12-11; processing...
report for period: 2022-11-28 --> 2022-12-11; completed


<ipython-input-63-87eb14e9ce30>:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table = table.append(df)


## Get Cumulative Sums and AUS-wide totals

In [64]:
states = ['act', 'nsw', 'nt','qld', 'sa', 'tas', 'vic', 'wa']

for state in states:
    table[state+'_cumulative_sum'] = table.groupby(['disease_code'])[state].cumsum()

#row -wise sum of the states
table['aus'] = table[states].sum(axis=1)
table['aus_cumulative_sum'] = table.groupby(['disease_code'])['aus'].cumsum()

In [65]:
#row -wise sum of the states
table['aus'] = table[states].sum(axis=1)
table['aus_cumulative_sum'] = table.groupby(['disease_code'])['aus'].cumsum()

## Inspect a disease

In [66]:
# view all the columns but with the start_date first
# see if latest dates were added
table[table['disease_code']==84][['start_date', 'end_date', 'act', 'nsw', 'nt',
       'qld', 'sa', 'tas', 'vic', 'wa','aus_cumulative_sum']] #+table.columns.to_list()

,start_date,end_date,act,nsw,nt,qld,sa,tas,vic,wa,aus_cumulative_sum
1491,2022-05-16,2022-05-29,0.0,1.0,0,0,0,0.0,1.0,0,2.0
1492,2022-05-30,2022-06-12,0.0,3.0,0,0,0,0.0,2.0,0,7.0
1493,2022-06-13,2022-06-26,0.0,1.0,0,0,0,0.0,1.0,0,9.0
1494,2022-06-27,2022-07-10,2.0,10.0,0,1,1,0.0,4.0,0,27.0
1495,2022-07-11,2022-07-24,0.0,8.0,0,0,0,0.0,8.0,0,43.0
1496,2022-07-25,2022-08-07,0.0,8.0,0,1,0,0.0,9.0,1,62.0
1497,2022-08-08,2022-08-21,0.0,8.0,0,1,1,0.0,18.0,3,93.0
1498,2022-08-22,2022-09-04,0.0,9.0,0,0,0,0.0,19.0,1,122.0
1499,2022-09-05,2022-09-18,1.0,3.0,0,1,0,0.0,1.0,1,129.0
1500,2022-09-19,2022-10-02,0.0,0.0,0,1,0,0.0,1.0,1,132.0


### Sort table & save file

In [67]:
table.sort_values(by=['disease_code', 'start_date'], inplace=True)

In [68]:
table.to_csv('nndss.csv', index=False)